In [245]:
import pandas as pd
#from sqlalchemy import create_engine
import sqlite3
from sqlite3 import Error


In [247]:
#engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
try:
    engine = sqlite3.connect('sqlite_db_pythonsqlite.db')
except Error as e:
    print(e)

In [248]:
pd.read_sql_query('select * from Facilities',engine)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


Q10:Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 

In [249]:
df10=pd.read_sql_query('select f.name, sum(subquery.price*subquery.slots) as revenue \
                  from (select Bookings.facid,memid,slots,\
                  case when memid = 0 THEN guestcost\
                  else membercost end as price\
                  from Bookings\
                  left join Facilities\
                  using(facid))as subquery\
                  left join Facilities as f\
                  using(facid)\
                  group by facid\
                  HAVING revenue < 1000\
                  order by revenue',engine)
df10

,name,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


 Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 

In [250]:
recommended = pd.read_sql_query('SELECT  a.surname,a.firstname, b.firstname || " " || b.surname recommended_by\
                  from Members a, Members b\
                  where a.recommendedby = b.memid\
                  order by a.surname,a.firstname ASC',engine)
recommended

,surname,firstname,recommended_by
0,Bader,Florence,Ponder Stibbons
1,Baker,Anne,Ponder Stibbons
2,Baker,Timothy,Jemima Farrell
3,Boothe,Tim,Tim Rownam
4,Butters,Gerald,Darren Smith
5,Coplin,Joan,Timothy Baker
6,Crumpet,Erica,Tracy Smith
7,Dare,Nancy,Janice Joplette
8,Genting,Matthew,Gerald Butters
9,Hunt,John,Millicent Purview


Q12: Find the facilities with their usage by member, but not guests 

In [229]:
df12 = pd.read_sql_query('with by_facility as (select sum(slots) as total_slots,facid \
                 from Bookings\
                 where memid != 0\
                 group by facid)\
                 select f.name as facility_name, bf.total_slots as total_member_usage_by_slot\
                 from Facilities as f\
                  left join by_facility as bf\
                  using (facid)'
                 ,engine)
df12

,facility_name,total_member_usage_by_slot
0,Tennis Court 1,957
1,Tennis Court 2,882
2,Badminton Court,1086
3,Table Tennis,794
4,Massage Room 1,884
5,Massage Room 2,54
6,Squash Court,418
7,Snooker Table,860
8,Pool Table,856


Q13: Find the facilities usage by month, but not guests 

In [234]:
df13 = pd.read_sql_query('SELECT Facilities.name, strftime("%m",date(starttime)) AS MONTH , SUM( slots ) AS total_slots_booked\
                    FROM Bookings\
                    left join Facilities\
                    using(facid)\
                    where Bookings.memid !=0\
                    GROUP BY MONTH , Facilities.name',engine)
df13

,name,MONTH,total_slots_booked
0,Badminton Court,07,165
1,Massage Room 1,07,166
2,Massage Room 2,07,8
3,Pool Table,07,110
4,Snooker Table,07,140
5,Squash Court,07,50
6,Table Tennis,07,98
7,Tennis Court 1,07,201
8,Tennis Court 2,07,123
9,Badminton Court,08,414
